# GFS Forecast Maps from Thredds Server via NCSS and Siphon
# 500mb Vorticity and Heights

## Justin Richling
## 11/15/18

https://doi.org/10.6084/m9.figshare.5244637.v1

In [1]:
# System Tools
import os

# Importing Datetime Libraries
from datetime import datetime, timedelta

# CartoPy Map Plotting Libraires
import cartopy.crs as ccrs
import cartopy.feature as cfeature

# Numerical and Scientific Libraries
import numpy as np
from scipy.ndimage import gaussian_filter

# Accessing Data from XLM Catalog via Siphon Libraries
from siphon.catalog import TDSCatalog
from siphon.ncss import NCSS

# MetPy Libraries
from metpy.plots import add_metpy_logo

# NetCDF Libraries
from netCDF4 import num2date

# Matplotlib Plotting Libraries
import matplotlib.pyplot as plt
from matplotlib import patheffects

# Warnings
import warnings
warnings.filterwarnings('ignore')

from Vorticity_Color_Bar import vort_cmap

In [2]:
# Set the font 
font = {'family': 'serif',
        'color':  'darkred',
        'weight': 'normal',
        'size': 18,
        }

## Helper Functions

In [3]:
# Thanks to the crew over at Metpy for this handy little function
def find_time_var(var, time_basename='time'):
    for coord_name in var.coordinates.split():
        if coord_name.startswith(time_basename):
            return coord_name
    raise ValueError('No time variable found for ' + var.name)

<h2>----------------------------------------------//---------------------------------------------------------</h2>

## Set the Map Projection

In [4]:
# Set Projection of Data
datacrs = ccrs.PlateCarree()

# Set Projection of Plot
plotcrs = ccrs.LambertConformal(central_latitude=[30, 60], central_longitude=-100)

# Add Map Features
states_provinces = cfeature.NaturalEarthFeature(category='cultural',
    name='admin_1_states_provinces_lakes',scale='50m', facecolor='none')

country_borders = cfeature.NaturalEarthFeature(category='cultural',
    name='admin_0_countries',scale='50m', facecolor='none')

# Colorbar Axis Placement (under figure)
colorbar_axis = [0.183, 0.09, 0.659, 0.03] # [left, bottom, width, height]

# Lat/Lon Extents [lon0,lon1,lat0,lat1]
extent = [-130., -70, 20., 60.]

<h2>----------------------------------------------//---------------------------------------------------------</h2>

## Figure out where the saved maps will go

In [5]:
now = datetime.utcnow()
#now = datetime(2019,4,10,0,0)
today_day = int('{0:%d}'.format(now))
today_year = int('{0:%Y}'.format(now))
today_month = int('{0:%m}'.format(now))
print(today_day,today_year,today_month)


# Set a path to save the plots with string format for the date to set the month and day
im_save_path ="/path/to/saved/images/"
im_save_path =f"/Users/chowdahead/Desktop/Weather_Blog/{today_year}/{today_month}_{today_day}/"
print(im_save_path)

# Check to see if the folder already exists, if not create it
if not os.path.isdir(im_save_path):
    os.makedirs(im_save_path)

# Uncomment if you want to automatically change to the map folder    
#os.chdir(im_save_path)

19 2019 11
/Users/chowdahead/Desktop/Weather_Blog/2019/11_19/


<h2>----------------------------------------------//---------------------------------------------------------</h2>

<h1><font><center>-- 500mb Vorticity and Heights --</center></font></h1>

## Start at the top of the thredds catalog

In [6]:
# Request the GFS data from the thredds server
gfs = TDSCatalog('http://thredds-jetstream.unidata.ucar.edu/thredds/catalog/grib/'
                 'NCEP/GFS/CONUS_20km/catalog.xml')

dataset = list(gfs.datasets.values())[1]
#print(dataset.access_urls)

# Create NCSS object to access the NetcdfSubset
ncss = NCSS(dataset.access_urls['NetcdfSubset'])

# get current date and time
#now = forecast_times[0]
now = datetime(today_year,today_month,today_day,0,0)
# define time range you want the data for
start = now
print(start)
delt_t = 48
end = now + timedelta(hours=delt_t)

query = ncss.query()
query.time_range(start, end)
Lat = query.lonlat_box(north=60, south=20, east=310, west=230)
query.accept('netcdf4')
query.variables("Absolute_vorticity_isobaric",'Geopotential_height_isobaric').add_lonlat(True)


# Request data for the variables you want to use
data = ncss.get_data(query)

# Get time into a datetime object
time_var = data.variables[find_time_var(data.variables["Absolute_vorticity_isobaric"])]
time_var = num2date(time_var[:], time_var.units).tolist()
time_strings = [t.strftime('%m/%d %H:%M') for t in time_var]

time_var = data.variables[find_time_var(data.variables["Absolute_vorticity_isobaric"])]
time_final = num2date(time_var[:].squeeze(), time_var.units)

vort_name = "Absolute_vorticity_isobaric"
hgt_name = "Geopotential_height_isobaric"

vort = data.variables[vort_name][:]
hgt = data.variables[hgt_name][:]

print("done.")

2019-11-19 00:00:00
done.


In [7]:
# Pull out the lat and lon data
lats = data.variables['lat'][:]
lons = data.variables['lon'][:]

# Combine 1D latitude and longitudes into a 2D grid of locations
#lon_2d, lat_2d = np.meshgrid(lons, lats)

In [18]:
def Vort_500(time_index):

                                # Setup Contour Label Options
#---------------------------------------------------------------------------------------------------    
    kw_clabels = {'fontsize': 11, 'inline': True, 'inline_spacing': 5, 'fmt': '%i',
              'rightside_up': True, 'use_clabeltext': True}

                                    # Setup Figure
#---------------------------------------------------------------------------------------------------    
    fig = plt.figure(figsize=(17., 11.))

    add_metpy_logo(fig, 25, 950, size='small')

                                    # Add the Map 
#---------------------------------------------------------------------------------------------------
    ax = plt.subplot(111, projection=plotcrs)

# Set extent and plot map lines
    ax.set_extent(extent, datacrs)
    
    ax.coastlines(resolution='50m')

                            # Add State/Country Boundaries to Plot
#---------------------------------------------------------------------------------------------------    
    state_borders = cfeature.NaturalEarthFeature(
                category='cultural', name='admin_1_states_provinces_lines',
                scale='50m', facecolor='none')
    ax.add_feature(state_borders, edgecolor='b', linewidth=1, zorder=3)
    
    country_borders = cfeature.NaturalEarthFeature(category='cultural',
                name='admin_0_countries',scale='50m', facecolor='none')
    ax.add_feature(country_borders,edgecolor='b',linewidth=0.2)
    
    ax.add_feature(cfeature.STATES)
    ax.add_feature(cfeature.BORDERS)
        
                                    # File and Title Times
#---------------------------------------------------------------------------------------------------
    # Time index for data variables
    time = time_strings[time_index]
    
    # Set string for saved image file name
    file_time = str(time_final[0]).replace("-","_").replace(" ","_").replace(":","")[:-2]+"Z"
    
    # Set forecast date and hour  
    forecast_date = "{}".format(today_year)+'-'+time_strings[time_index].replace("/","-")[:-5]
    forecast_hour = time_strings[time_index][-5:]+"Z"
    
    # Set initialization date and hour 
    init_date = "{}".format(today_year)+'-'+time_strings[0].replace("/","-")[:-5]
    init_hour = time_strings[0].replace("/","-")[-5:]+"Z"
    
    
                                        # Plot Title
#---------------------------------------------------------------------------------------------------
    ax.set_title('GFS 20km CONUS: 500mb Heights and Abs Vorticity '+ "("+"$\mathregular{s^{-1}}$"+")", 
                 size=16, loc='left',fontdict=font)
    
    ax.set_title(f"Init Hour: {init_date} {init_hour}\nForecast Hour: {forecast_date} {forecast_hour}",
                 size=16, loc='right',fontdict=font)

    
                                        # 500mb Heights
#---------------------------------------------------------------------------------------------------
    hgt_500 = hgt[time_strings.index(time),data.variables["isobaric1"][:].tolist().index(50000),:,:]

    clev500 = np.arange(5200, 6000, 60)
    cs = ax.contour(lons, lats, hgt_500,clev500 ,colors='black', linewidths=2.0,
                    linestyles='solid', transform=ccrs.PlateCarree())
    plt.clabel(cs, **kw_clabels)

                                        # Vorticity
#---------------------------------------------------------------------------------------------------
    vort_500 = vort[time_strings.index(time),data.variables["isobaric"][:].tolist().index(50000),:,:]

    vort_levels = np.arange(-.00055,.0007,0.00001)
    cs2 = ax.contourf(lons, lats, vort_500,vort_levels,
                     transform=datacrs,cmap=vort_cmap)
    
    cbaxes = fig.add_axes(colorbar_axis) # [left, bottom, width, height]
    cbar = plt.colorbar(cs2, orientation='horizontal',cax=cbaxes)
    
    #cbar.set_label(r'$s{^-1}$')
    #plt.clabel(cs2, **kw_clabels)

                                        # Save Figure
#---------------------------------------------------------------------------------------------------    
    VORT = im_save_path+"GFS/Vorticity/"
    if not os.path.isdir(VORT):
        os.makedirs(VORT)
        
    time_index *= 3
    if time_index < 10:
        times = f"0{time_index}"
    else:
        times = f"{time_index}"

    outfile = f"{VORT}GFS_20km_Vort_Heights_500mb_{file_time}_F{times}.png"
    fig.savefig(outfile,bbox_inches='tight',dpi=120)
    plt.close(fig)

In [19]:
for i in range(0,int(delt_t/3)):
    Vort_500(i)
    print("next.")
print("All done.")

next.
next.
next.
next.
next.
next.
next.
next.
next.
next.
next.
next.
next.
next.
next.
next.
All done.
